
# Generative AI with Databricks

## From Predictive to Prescriptive Maintenance
Manufacturers face labor shortages, supply chain disruptions, and rising costs, making efficient maintenance essential. Despite investments in maintenance programs, many struggle to boost asset productivity due to technician shortages and poor knowledge-sharing systems. This leads to knowledge loss and operational inefficiencies.

<div style="font-family: 'DM Sans';">
  <div style="width: 400px; color: #1b3139; margin-left: 50px; margin-right: 50px; float: left;">
    <div style="color: #ff5f46; font-size:50px;">73%</div>
    <div style="font-size:25px; margin-top: -20px; line-height: 30px;">
      of manufacturers struggle to recruit maintenance technicians — McKinsey (2023)
    </div>
    <div style="color: #ff5f46; font-size:50px;">55%</div>
    <div style="font-size:25px; margin-top: -20px; line-height: 30px;">
      of manufacturers lack formal knowledge-sharing systems — McKinsey (2023)
    </div>
  </div>
</div>

Generative AI can transform maintenance by reducing downtime and improving productivity. While predictive maintenance anticipates failures, Generative AI enables prescriptive maintenance. Using historical data, AI systems can identify issues, generate solutions, and assist technicians, allowing junior staff to perform effectively and freeing experts for complex tasks.
<br><br>

### From Models to Agent Systems
Generative AI is moving from standalone models to modular agent systems ([Zaharia et al., 2024](https://bair.berkeley.edu/blog/2024/02/18/compound-ai-systems/)). These systems integrate retrievers, models, prompts, and tools to handle complex tasks. Their modular design allows seamless upgrades (e.g., integrating a new LLM) and adaptation to changing needs.

<br><br>
<img style="float: right; margin-top: 10px;" width="700px" src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/manufacturing/lakehouse-iot-turbine/team_flow_liza.png" />


<br>
<div style="font-size: 19px; margin-left: 0px; clear: left; padding-top: 10px; ">
<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/liza.png" style="float: left;" width="80px"> 
<h3 style="padding: 10px 0px 0px 5px;">Liza, a Generative AI engineer, uses the Databricks Intelligence Platform to:</h3>
<ul style="list-style: none; padding: 0; margin-left: 05%;">
  <li style="margin-bottom: 10px; display: flex; align-items: center;">
    <div class="badge" style="height: 30px; width: 30px; border-radius: 50%; background: #fcba33; color: white; text-align: center; line-height: 30px; font-weight: bold; margin-right: 10px;">1</div>
    Build real-time data pipelines
  </li>
  <li style="margin-bottom: 10px; display: flex; align-items: center;">
    <div class="badge" style="height: 30px; width: 30px; border-radius: 50%; background: #fcba33; color: white; text-align: center; line-height: 30px; font-weight: bold; margin-right: 10px;">2</div>
    Retrieve vectors & features
  </li>
  <li style="margin-bottom: 10px; display: flex; align-items: center;">
    <div class="badge" style="height: 30px; width: 30px; border-radius: 50%; background: #fcba33; color: white; text-align: center; line-height: 30px; font-weight: bold; margin-right: 10px;">3</div>
    Create AI agent tools
  </li>
  <li style="margin-bottom: 10px; display: flex; align-items: center;">
    <div class="badge" style="height: 30px; width: 30px; border-radius: 50%; background: #fcba33; color: white; text-align: center; line-height: 30px; font-weight: bold; margin-right: 10px;">4</div>
    Build & deploy agents
  </li>
  <li style="margin-bottom: 10px; display: flex; align-items: center;">
    <div class="badge" style="height: 30px; width: 30px; border-radius: 50%; background: #fcba33; color: white; text-align: center; line-height: 30px; font-weight: bold; margin-right: 10px;">5</div>
    Operate in batch or real-time
  </li>
  <li style="display: flex; align-items: center;">
    <div class="badge" style="height: 30px; width: 30px; border-radius: 50%; background: #fcba33; color: white; text-align: center; line-height: 30px; font-weight: bold; margin-right: 10px;">6</div>
    Evaluate agent performance
  </li>
</ul>
</div>

**Databricks empowers Liza with a Data + AI platform for Prescriptive Maintenance.** Let’s explore how to deploy this in production.

<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=lakehouse&org_id=4003492105941350&notebook=%2F05-Generative-AI%2F05.1-ai-tools-iot-turbine-prescriptive-maintenance&demo_name=lakehouse-iot-platform&event=VIEW&path=%2F_dbdemos%2Flakehouse%2Flakehouse-iot-platform%2F05-Generative-AI%2F05.1-ai-tools-iot-turbine-prescriptive-maintenance&version=1">

## Building Agent Systems with Databricks Mosaic AI agent framework

We will build an Agent System designed to generate prescriptive work orders for wind turbine maintenance technicians. This system integrates multiple interacting components to ensure proactive and efficient maintenance, thereby optimizing the overall equipment effectiveness.

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/manufacturing/lakehouse-iot-turbine/iot_agent_graph_v2_0.png" style="margin-left: 5px; float: right"  width="1000px;">

Databricks simplifies this by providing a built-in service to:

- Create and store your AI tools leveraging UC functions
- Execute the AI tools in a safe way
- Use agents to reason about the tools you selected and chain them together to properly answer your question. 


This notebook creates the three Mosaic AI tools and associated Mosaic AI endpoints, which will be composed together into a agent in notebook [05.2-agent-creation-guide]($./05.2-agent-creation-guide).
1. **Turbine predictor** which uses a Model Serving endpoint to predict turbines at risk of failure.
2. **Turbine specifications retriever** which retrieve the turbine specifications based on its id.
3. **Turbine maintenance guide**  which uses a Vector Search endpoint to retrieve maintenance guide based on the turbines and issues being adressed.

In [0]:
%pip install databricks-vectorsearch==0.49 databricks-feature-engineering==0.8.0 databricks-sdk==0.40.0 
dbutils.library.restartPython()

In [0]:
%run ../_resources/00-setup $reset_all_data=false

In [0]:
%sql
DROP FUNCTION IF EXISTS turbine_specifications_retriever;

--turbine_specifications_retriever to get the current status of a turbine
--This function is used to retrieve the turbine specifications based on its id

CREATE OR REPLACE FUNCTION 
turbine_specifications_retriever(turbine_id STRING COMMENT 'ID of the wind turbine to look up')
RETURNS TABLE (
  avg_energy DOUBLE COMMENT 'Average energy reading',
  std_sensor_A DOUBLE COMMENT 'Sensor A reading',
  std_sensor_B DOUBLE COMMENT 'Sensor B reading',
  std_sensor_C DOUBLE COMMENT 'Sensor C reading',
  std_sensor_D DOUBLE COMMENT 'Sensor D reading',
  std_sensor_E DOUBLE COMMENT 'Sensor E reading',
  std_sensor_F DOUBLE COMMENT 'Sensor F reading'
)
LANGUAGE SQL
COMMENT 'This function retrieves the turbine sensor readings / specifications based on the turbine_id'
RETURN
(
SELECT 

avg_energy, std_sensor_A, std_sensor_B, std_sensor_C, std_sensor_D, std_sensor_E, std_sensor_F
FROM main.dbdemos_iot_turbine.turbine_current_features
WHERE turbine_id = turbine_specifications_retriever.turbine_id
SORT BY hourly_timestamp DESC
limit 1
);

In [0]:
%sql
SELECT * FROM turbine_specifications_retriever('004a641f-e9e5-9fff-d421-1bf88319420b')



## Part 1: Create the Turbine Predictor as a tool to predict turbine failure

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/manufacturing/lakehouse-iot-turbine/iot_agent_graph_v2_1.png" style="float: right; width: 600px; margin-left: 10px">

To enable our Agent System to predict turbine failtures based on industrial IoT sensor readings, we will rely on the model we deployed previously in the  [./04.3-running-inference-iot-turbine]($./04.3-running-inference-iot-turbine) notebook. 

**Make sure you run this ML notebook to create the model serving endpoint!**


### Using the Model Serving as tool to predict faulty turbines
Let's define the turbine predictor tool function our LLM agent will be able to execute. 

AI agents use [AI Agent Tools](https://docs.databricks.com/en/generative-ai/create-log-agent.html#create-ai-agent-tools) to perform actions besides language generation, for example to retrieve structured or unstructured data, execute code, or talk to remote services (e.g. send an email or Slack message). 

These functions can contain any logic, from simple SQL to advanced python. Below we wrap the model serving endpoint in a SQL function using '[ai_query function](https://docs.databricks.com/en/sql/language-manual/functions/ai_query.html)'.

In [0]:
%sql
DROP FUNCTION IF EXISTS turbine_maintenance_predictor;

CREATE OR REPLACE FUNCTION 
turbine_maintenance_predictor(
    sensor_values ARRAY<DOUBLE> COMMENT 'Array of 7 double values (energy and sensor readings) from a wind turbine. Example: array(avg_energy, std_sensor_A, std_sensor_B, std_sensor_C, std_sensor_D, std_sensor_E, std_sensor_F)'
    )
RETURNS STRING
LANGUAGE SQL
COMMENT 'This tool predicts whether or not a turbine is faulty to facilitate proactive maintenance. It expects an array of 7 double values (energy and sensor readings) as input and returns a string indicating if a particular sensor is predicted to be faulty or if all sensors are ok.'
RETURN
(
    SELECT CASE 
        -- Convert the prediction to a string that can be understood by the agent based on the output of the AI query
        WHEN prediction = '0.0' THEN 'Sensor F fault'
        WHEN prediction = '1.0' THEN 'ok' 
        WHEN prediction = '2.0' THEN 'Sensor B fault' 
        WHEN prediction = '3.0' THEN 'Sensor D fault'
        ELSE 'faulty'
        END AS prediction
    FROM (    
        SELECT ai_query(
            'dbdemos_iot_turbine_prediction_endpoint', 
            sensor_values,
            'DOUBLE'
        ) as prediction
    )    
);

In [0]:
# %sql
# DROP FUNCTION IF EXISTS turbine_maintenance_predictor;

# CREATE OR REPLACE FUNCTION 

# turbine_maintenance_predictor(
#     sensor_values ARRAY<DOUBLE>
#     )
# RETURNS STRING
# LANGUAGE SQL
# COMMENT 'This tool predicts whether or not a turbine is faulty to facilitate proactive maintenance. It expects an array of 7 double values (energy and sensor readings) as input and returns a string'
# RETURN
# (
#     SELECT ai_query(
#         'dbdemos_iot_turbine_prediction_endpoint', 
        
#         array(
#             sensor_values[0],
#             sensor_values[1],
#             sensor_values[2],
#             sensor_values[3],
#             sensor_values[4],
#             sensor_values[5],
#             sensor_values[6]
#             ),
#         'DOUBLE'
#     )
# );

In [0]:
# %sql
# -- for testing purposes only
# DROP FUNCTION IF EXISTS get_dummy_sensor_value_readings;

# CREATE OR REPLACE FUNCTION get_dummy_sensor_value_readings()
# RETURNS ARRAY<DOUBLE>
# LANGUAGE SQL
# COMMENT 'Returns an array of 7 random double values simulating sensor readings'
# RETURN (
#   array(
#     rand(), 
#     rand(), 
#     rand(), 
#     rand(), 
#     rand(), 
#     rand(), 
#     rand()
#   )
# );

In [0]:
# df = spark.sql("SELECT get_dummy_sensor_value_readings() AS sensor_values")
# display(df)

Agent should call turbine_specifications_retriever() to get sensor readings, then agent needs to format readings as an array and call turbine_maintenance_predictor() to get a prediction

Now we can test out our function below:

In [0]:
%sql
-- test our prediction function
SELECT turbine_maintenance_predictor(
    array(
      0.9000803742589635,                           -- avg_energy
      2.2081154200781867,                           -- std_sensor_A
      2.6012126574143823,                           -- std_sensor_B
      2.1075958066966423,                           -- std_sensor_C
      2.2081154200781867,                           -- std_sensor_D
      2.6012126574143823,                           -- std_sensor_E
      2.1075958066966423                            -- std_sensor_F
    )  
) AS prediction

## Part 3: Add a tool to access our maintenance guide content and provide support to the operator during maintenance operation

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/manufacturing/lakehouse-iot-turbine/iot_agent_graph_v2_3.png" style="float: right; width: 600px; margin-left: 10px">


We were provided with PDF guide containing all the error code and maintenance steps for the critical components of our wind turbine. The're saved as pdf file in our volume.

Let's parse them and index them so that we can properly retrieve them. We'll save them in a Vector Search endpoint and leverage it to guide the operators with the maintenance step and recommendations.

We'll use a Managed embedding index to make it simple. In this section we will:

1. Parse and save our PDF text in a Delta Table using Databricks AI Query `ai_parse_document`
2. Create a `Vector Search endpoint` (required to host your vector search index)
3. Create a `Vector Search Direct Index`  (the actual index)
4. Create a `Tool (UC function)` using our vector search 


### 2.1. Parse and save our PDF text
Let's start by parsing the maintenance guide documents, saved as pdf in our volume:

In [0]:
%sql
DROP TABLE IF EXISTS turbine_maintenance_guide;

CREATE TABLE turbine_maintenance_guide (
  id BIGINT GENERATED ALWAYS AS IDENTITY,
  EAN STRING,
  weight STRING,
  component_type STRING,
  component_name STRING,
  full_guide STRING)
  TBLPROPERTIES (delta.enableChangeDataFeed = true);

In [0]:
# df_selected = spark.read.option("header", "true").csv("/Volumes/main/dbdemos_iot_turbine/additional_data/pdf_to_structured.csv").select("EAN", "weight", "component_type", "component_name", "full_guide").na.drop(how="all")

# df_selected.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("main.dbdemos_iot_turbine.turbine_maintenance_guide")

In [0]:
%sql
INSERT INTO turbine_maintenance_guide (EAN, weight, component_type, component_name, full_guide)

SELECT ai_query(
    'databricks-gemma-3-12b',
    CONCAT("Extract the EAN from the following text. Return only the EAN. \n\nText:", full_guide)    -- Placeholder for the prompt and input
    ) AS EAN  -- Placeholder for the output column
    ,ai_query(
    'databricks-gemma-3-12b',
    CONCAT("Extract the component weight from the following text. Return only the component weight. \n\nText:", full_guide)    -- Placeholder for the prompt and input
    ) AS weight  -- Placeholder for the output column
    ,ai_query(
    'databricks-gemma-3-12b',
    CONCAT("Extract the component type from the following text. Return only the component type. \n\nText:", full_guide)    -- Placeholder for the prompt and input
    ) AS component_type  -- Placeholder for the output column 
    ,ai_query(
    'databricks-gemma-3-12b',
    CONCAT("Extract the component name from the following text. Return only the component name. \n\nText:", full_guide)    -- Placeholder for the prompt and input
    ) AS component_name  -- Placeholder for the output column
    ,full_guide 
FROM (
    -- Combine the content of all pages into a single string separated by new lines
    SELECT array_join(

            -- Transform each page struct in the array to just its 'content' field (extract text from each page)
            transform(
              parsed_document:document.pages::ARRAY<STRUCT<content:STRING>>, -- Array of page structs from the parsed document
              x -> x.content -- For each struct (page), extract the 'content' string
             ), 
             
             '\n' -- Join all extracted page contents with a newline character as the separator
             ) 
             
             AS full_guide
    FROM (
      -- Parse the document content
      SELECT ai_parse_document(content) AS parsed_document
      FROM READ_FILES("/Volumes/main/dbdemos_iot_turbine/turbine_raw_landing/maintenance_guide", format => 'binaryFile')
    )
);

SELECT * FROM turbine_maintenance_guide

### 2.2. Creating the Vector Search endpoint

Let's create a new Vector search endpoint. You can also use the [UI under Compute](#/setting/clusters/vector-search) to directly create your endpoint.

In [0]:
from databricks.vector_search.client import VectorSearchClient
vsc = VectorSearchClient(disable_notice=True)

if not endpoint_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME):
    vsc.create_endpoint(name=VECTOR_SEARCH_ENDPOINT_NAME, 
                        endpoint_type="STANDARD")

wait_for_vs_endpoint_to_be_ready(vsc, 
                                 VECTOR_SEARCH_ENDPOINT_NAME)


print(f"Endpoint named {VECTOR_SEARCH_ENDPOINT_NAME} is ready.")


### 2.3 Creating the Vector Search Index

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/index_creation.gif?raw=true" width="600px" style="float: right; margin-left: 10px">

You can view your endpoint on the [Vector Search Endpoints UI](#/setting/clusters/vector-search). Click on the endpoint name to see all indexes that are served by the endpoint.

All we now have to do is to as Databricks to create the index on top of our table. The Delta Table will automatically be synched with the index.


Again, you can do that using your Unity Catalog UI, and selecting the turbine_maintenance_guide table in your Unity Catalog, and click on add a vector search. 

In [0]:
import databricks.sdk.service.catalog as c

# Where we want to store our index
vs_index_fullname = f"{catalog}.{db}.turbine_maintenance_guide_vs_index"

if not index_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname):
  print(f"Creating index {vs_index_fullname} on endpoint {VECTOR_SEARCH_ENDPOINT_NAME}...")
  
  index = vsc.create_delta_sync_index(
    endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
    source_table_name=f"{catalog}.{db}.turbine_maintenance_guide",
    index_name=vs_index_fullname,
    pipeline_type="TRIGGERED",
    primary_key='id',
    embedding_source_column="full_guide",
    embedding_model_endpoint_name="databricks-gte-large-en"
  )
else:
  print(f"Grabbing existing index {vs_index_fullname} on endpoint {VECTOR_SEARCH_ENDPOINT_NAME}...")
  index = vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)

In [0]:
exists = index_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)
display(spark.createDataFrame([(vs_index_fullname, exists)], ["index_name", "exists"]))

### 2.4 Create our tool
Below, we utilize the _VECTOR\_SEARCH_ SQL function from Databricks to easily set up our maintenance reports retriever function. Our agent will utilize this function in the subsequent steps!

In [0]:
spark.sql("DROP FUNCTION IF EXISTS turbine_maintenance_guide_retriever")
spark.sql(f"""
CREATE OR REPLACE FUNCTION 
turbine_maintenance_guide_retriever(question STRING COMMENT 'Question to be answered from the turbine maintenance guides.')
RETURNS ARRAY<STRING>
LANGUAGE SQL
COMMENT 'This tool searches / retrieves the wind turbine maintenance guide for a given question'
RETURN (
  SELECT collect_list(full_guide) 
  FROM VECTOR_SEARCH(index => '{catalog}.{schema}.turbine_maintenance_guide_vs_index', query => question, num_results => 1) ) """)

In [0]:
%sql 
-- Let's test the tool we created
SELECT turbine_maintenance_guide_retriever('The VibeGuard TVS-950 is giving me an error code TVS-001.') AS reports

## Exploring Mosaic AI Tools in Unity Catalog

Our tools are ready! 

You can now view the UC function tools in Catalog Explorer. Click **Catalog** in the sidebar. In the Catalog Explorer, navigate to your catalog and schema. 

The UC function tools appears under **Functions**. 

<img src="https://github.com/Datastohne/demo/blob/main/Screenshot%202024-09-18%20at%2016.24.24.png?raw=true"/>

---

In [0]:
# %pip install --quiet openmeteo-requests
# %pip install --quiet requests-cache retry-requests numpy pandas

In [0]:
# import openmeteo_requests

# import pandas as pd
# import requests_cache
# from retry_requests import retry

# # Setup the Open-Meteo API client with cache and retry on error
# cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
# retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
# openmeteo = openmeteo_requests.Client(session = retry_session)

# # Make sure all required weather variables are listed here
# # The order of variables in hourly or daily is important to assign them correctly below
# url = "https://api.open-meteo.com/v1/forecast"
# params = {
# 	"latitude": 52.52,
# 	"longitude": 13.41,
# 	"hourly": "temperature_2m"
# }
# responses = openmeteo.weather_api(url, params=params)


## What’s next: test your Agents with Databricks Playground

Now that we have our AI Tools ready and registered in Unity Catalog, we can compose them into an agent system that generates maintenance work orders using the Mosaic AI agent framework.

Open the [05.2-agent-creation-guide]($./05.2-agent-creation-guide) notebook to create and deploy the system.